# Uczenie głębokie cz. II

Ten notatnik ma na celu przedstawienie sposobów wykorzystania uczenia głębokiego w różnych zastosowaniach. W trakcie zadania najpierw dostosujemy istniejącą sieć konwolucyjną do nowych danych, zobaczymy jak stosować sieci do rekomendowania produktów oraz jak nauczyć klasyfikator tekstowy. Na tych zajęciach wykorzystamy bibliotekę [pytorch-lightning](https://lightning.ai/docs/pytorch/stable/), która jest wrapperem na [PyTorch](https://pytorch.org/) oraz, tak jak ostatnio, [Keras](https://keras.io/). Poniższe skrypty są w istocie kompilacją różnych materiałów szkoleniowych do pytorch (w tym też torchvision i pytorch-geometric), pytorch-lightning i Kerasa.

Po wykonaniu tego zadania powinieneś:
+ wiedzieć dostosować istniejącą architekturę (wraz z wagami) do własnego problemu,
+ wiedzieć jak zamrażać warstwy sieci,
+ umieć zastosować sieć neuronową do danych tekstowych,
+ potrafić stworzyć sieć grafową.

## Przygotowanie środowiska

Ćwiczenie wykonamy na platformie [Google Colab](https://colab.research.google.com/), aby nie musieć instalować bibliotek i zbiorów danych lokalnie. W Colabie wybieramy runtime z GPU.

## Rozpoznawanie obrazów

Spróbujemy nauczyć sieć rozpoznającą typy terenu na zdjęciach satelitarnych. W tym celu wykorzystamy sieć nauczoną na zbiorze danych ImageNet, która umie klasyfikować pojedyncze obiekty na zdjęciach (psy, koty, czołgi, itp.). W naszym problemie będziemy musieli rozpoznać kilka rodzajów terenu na każdym zdjęciu (multi-label classification) i nie będą one przypominały obiektów ze zbioru ImageNet.

W zadaniu wykorzystamy pytorch-lightning, pytorch i torchvision. Pokażemy też, jak używać tensorboard z lightningiem.

In [ ]:
!pip install pytorch-lightning

In [ ]:
%load_ext tensorboard

### Dane

Dane pochodzą z zakończonego konkursu na Kaggle o nazwie [Planet: Understanding the Amazon from Space](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space). Najpierw pobierzmy dane:

In [ ]:
import os
os.makedirs('data/planet', exist_ok=True)
!wget "https://drive.google.com/u/3/uc?id=1doTIfs4q9zxINS4WKdypd15p6y7VN02Y&export=download&confirm=yes" -O data/planet/train_v2.csv
!wget "https://drive.usercontent.google.com/download?id=1DIJhHbmKX0wZk4I1e8mmhA4Pss-xPPBK&export=download&confirm=yes" -O data/planet/train-jpg.zip
!unzip data/planet/train-jpg.zip -d data/planet/

Jeżeli pobieranie się udało zobaczmy jak wyglądają przykładowe zdjęcia satelitarne.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def plots_from_files(list_paths, titles=None, maintitle=None, figsize=(10, 5)):
  num_images = len(list_paths)
  fig, axs = plt.subplots(1, num_images, figsize=figsize)
  if maintitle:
      fig.suptitle(maintitle, fontsize=16)
  if num_images == 1:
      axs = [axs]  # Ensure axs is iterable when there's a single image
  for ax, img_path, title in zip(axs, list_paths, titles):
      img = mpimg.imread(img_path)
      ax.imshow(img)
      ax.axis('off')
      if title:
          ax.set_title(title, fontsize=10)
  plt.tight_layout()
  plt.show()


list_paths = [f"data/planet/train-jpg/train_0.jpg", f"data/planet/train-jpg/train_1.jpg"]
titles=["haze primary", "agriculture clear primary water"]

plots_from_files(list_paths, titles=titles, maintitle="Multi-label classification")


Zdjęcie po lewej ma etykiety (klasy) *haze* i *primary*, a zdjęcie po prawej ma etykiety *agriculture*, *clear*, *primary* i *water*.

### Ładowanie danych do sieci

W pierwszej kolejności ustalimy sposób ładowania danych do sieci i transformacje, któych chcemy użyć.

Najpierw zapoznajmy się z plikiem zawierającym etykiety.

In [ ]:
import pandas as pd

label_csv = f'data/planet/train_v2.csv' # plik csv z połączeniem nazwa_obrazu-etykiety
label_csv = pd.read_csv(label_csv)
label_csv.head()

Aby móc pracować z naszym datasetem w pytorchu, musimy zaimplementować specjalną strukturę (Dataset), która obsłuży czytanie plików i łączenie ich z etykietami.

Tutaj też będziemy podawać nasze transformacje, które chcemy zaaplikować do obrazów.

**Zadanie 1.1: Zaencoduj etykiety, tak aby zamiast aktualnej formy (tj. zwykłego stringa, np. "haze primary") otrzymać wektor z 1, jeśli dany obraz posiada daną etykietę i 0, jeśli nie posiada (zakładając, że w zbiorze są trzy etykiety, np. "haze", "primary", "clear" i encodujemy jest w takiej kolejności, dla przykładu "haze primary" powinniśmy otrzymać [1, 1, 0])**

Odpowiedni import jest już zrobiony - zobacz, czego możesz użyć.

In [ ]:
import os
import torch
from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import Dataset
import torch.nn.functional as F
from PIL import Image

class PlanetDataset(Dataset):
  def __init__(self, csv_file: str, root_dir: str, transform=None):
    data = pd.read_csv(csv_file)
    self.root_dir = root_dir

    data = self._filter_data(data)
    self.data_files = data['image_name'].to_list()

    #encodowanie etykiet - użyj data['tags'] i zesplituj, żeby uzyskać listy
    labels = ...

    self.mlb = ...
    labels = ...
    self.n_classes = ...
    self.labels = torch.Tensor(labels)

    self.transform = transform

  def _filter_data(self, data: pd.DataFrame):
    # funkcja do odrzucenia wierszy z csvki, do któych nie ma obrazów w naszych danych
    data_files = [i.split('.')[0] for i in os.listdir(self.root_dir)]
    correct_files = data.loc[:, 'image_name'].isin(data_files)
    return data.loc[correct_files, :]

  def __len__(self):
    return len(self.data_files)

  def __getitem__(self, idx):
    # zwraca obraz i jego etykietę, aplikuje transformacje do obrazu, jęsli zostały podane
    label = self.labels[idx, :]
    img_name = os.path.join(self.root_dir, f"{self.data_files[idx]}.jpg")
    image = Image.open(img_name).convert('RGB')

    if self.transform:
        image = self.transform(image)
    return image, label

Jak wspominałam na zajęciach, pytorch-lightning jest pewną formą uprządkowania kodu pytorcha. Szcególnie ważne są tutaj dwie struktury: `LightningDataModule` i `LightningModule`. Najpierw omówmy ten pierwszy. `LightningDataModule` zajmuje się obsługą danych, podziałem danych na zbiory treningowy i testowy, a także zadeklarowaniem DataLoaderów, które podzielą nasz Dataset na batche.

Poniżej implementujemy `PlanetDataModule`, czyli `LightningDataModule` dedykowany dla naszego zbioru. To tutaj m.in. określamy transformacje, które zostaną zaaplikowane do naszych obrazów.

Do zbioru treningowego chcemy jednak użyć nieco innych transformacji niż do zbiorów walidacyjnego i testowego.

**Zadanie 1.2: Używając torchvision (zaimportowany moduł v2) dodaj następujące transformacje do `self.train_transform`:**

* random crop
* odbicie (horizontal, z prawdopodobieństwem 0.5)
* rotacja (15 stopni)


In [ ]:
import pytorch_lightning as L
from pytorch_lightning import seed_everything
from torch.utils.data import random_split, DataLoader
from torchvision.transforms import v2

class PlanetDataModule(L.LightningDataModule):
  def __init__(self, batch_size=32, train_size=0.8, val_size=0.1, test_size=0.1, seed=42, num_workers=3):
    super().__init__()
    seed_everything(seed, workers=True)
    self.batch_size = batch_size
    self.train_size = train_size
    self.val_size = val_size
    self.test_size = test_size
    self.num_workers = num_workers
    self.generator = torch.Generator().manual_seed(seed)

    self.train_transform = v2.Compose([
        v2.ToImage(),
        v2.Resize((64, 64)),
        ...,        #tutaj dodaj potrzebne transformacje
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    self.val_test_transform = v2.Compose([
        v2.ToImage(),
        v2.Resize((64, 64)),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    self.setup()

  def setup(self, stage=None):
    dataset = PlanetDataset('data/planet/train_v2.csv', 'data/planet/train-jpg')
    train_size = int(self.train_size * len(dataset))
    val_size = int(self.val_size * len(dataset))
    test_size = len(dataset) - train_size - val_size
    self.train_dataset, self.val_dataset, self.test_dataset = random_split(
        dataset, [train_size, val_size, test_size], generator=self.generator)

    self.train_dataset.dataset.transform = self.train_transform
    self.val_dataset.dataset.transform = self.val_test_transform
    self.test_dataset.dataset.transform = self.val_test_transform

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)


Przechodzimy do implementacji wcześniej wspominanego `LightningModule`, w którym zawrzemy cały kod obsługujący sam model.

Używamy pre-trenowanego Resneta18, dostępnego w ramach bilioteki torchvision. Następnie mrozimy wszystkie parametry modelu, oprócz warstw końcowych.

Zamieniamy warstwę fully-connected oryginalnego resneta, aby odpowiadała naszemu zadaniu klasyfikacyjnemu. Podczas uczenia, tylko wagi `model.fc` będą aktualizowane.

Ponadto wybieramy funkcję straty i metryki, które chcemy obliczać.
Jako funkcję straty stosujemy Binary Cross Entropy, a z metryk obliczamy F2-score.

Następnie implemetujemy poszczególne kroki uczenia - treningowy, walidacyjny i testowy.

**Zadanie 1.3: Na podstawie funkcji `training_step` zaimplementuj `validation_step`. Zaloguj loss i F2-score otrzymany podczas walidacji (logując je odpowiednio pod 'val_loss' i 'val_f2')**

In [ ]:
from torchvision.models import resnet18
from sklearn.metrics import fbeta_score

class PlanetModel(L.LightningModule):
  def __init__(self, num_classes):
    super().__init__()
    self.save_hyperparameters()
    model = resnet18(pretrained=True)

    for param in model.parameters():
      param.requires_grad = False

    model.fc = torch.nn.Sequential(
          torch.nn.Linear(model.fc.in_features, 512),
          torch.nn.ReLU(),
          torch.nn.Linear(512, num_classes),
          torch.nn.Sigmoid()
    )

    self.model = model
    self.criterion = torch.nn.BCELoss()

  def forward(self, x):
    return self.model(x)

  def _compute_f2(self, y_hat, y):
    y_hat = (y_hat > 0.5).int().cpu().numpy()
    y = y.int().cpu().numpy()
    f2 = fbeta_score(y, y_hat, beta=2, average='samples')
    return f2

  def training_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.forward(x)
    loss = self.criterion(y_hat, y)
    f2 = self._compute_f2(y_hat, y)
    self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    self.log('train_f2', f2, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    return loss

  def validation_step(self, batch, batch_idx):
    ...

  def test_step(self, batch, batch_idx):
    x, y = batch
    y_hat = self.forward(x)
    loss = self.criterion(y_hat, y)
    f2 = self._compute_f2(y_hat, y)
    self.log('test_loss', loss, logger=True)
    self.log('test_f2', f2, logger=True)

  def configure_optimizers(self):
    optimizer = torch.optim.Adadelta(self.parameters())
    return optimizer

Mając już zaimplementowane wszytskie elementy, musimu połaczyć je w całość. Do tego użyjemy struktury `Trainer` z pytorch-lightning.

Dodatkowo, użyjemy dwóch callbacków - do zapisu najlepszego modelu i do early stoppingu (wcześniejszego zatrzymywania się uczenia, gdy nie uzyskamy poprawy funkcji straty na zbiorze walidacyjnym przez ileś epok).

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

data = PlanetDataModule(batch_size=256, num_workers=2)

num_classes = data.train_dataset.dataset.n_classes

model = PlanetModel(num_classes=num_classes)

os.makedirs('data/planet/models', exist_ok=True)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='data/planet/models',
    filename="resnet18-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
    mode="min"
)

early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="min"
)

trainer = Trainer(max_epochs=20, callbacks=[checkpoint_callback, early_stopping_callback])
trainer.fit(model, datamodule=data)

Wytrenowany model możemy załadować z pliku i odpalić go na zbiorze testowym.

In [ ]:
model_path = 'data/planet/models/resnet18-epoch=10-val_loss=0.14.ckpt'

model = PlanetModel.load_from_checkpoint(model_path)
trainer.test(model, datamodule=data)

Wszystkie dotychczas zalogowane wyniki możemy zobaczyć w tensorboard.

In [ ]:
%tensorboard --logdir lightning_logs/

### Keras

Podobną metodologię, obejmującą generowanie sztucznych danych wraz z wykorzystaniem istniejącej architektury i wag, można oczywiście zaimplementować również w Kerasie. Krótki tutorial jak to zrobić znajdziesz na stronie: https://keras.io/guides/transfer_learning/.

## Sieci grafowe

Grafowe sieci neuronowe stanowią dynamicznie rozwijającą się dziedzinę w obszarze uczenia maszynowego, oferując narzędzia do analizy danych o strukturze grafowej. W odróżnieniu od tradycyjnych sieci neuronowych, które operują na danych w formie wektorów czy macierzy, GNN pozwalają efektywnie przetwarzać i wyciągać informacje z danych, w których relacje pomiędzy elementami są równie istotne co same elementy. Przykłady takich danych obejmują sieci społeczne, grafy molekularne, sieci komunikacyjne czy systemy rekomendacji.

My skupimy się na grafach molekularnych, a konkretniej na zbiorze [Proteins](https://chrsmrrs.github.io/datasets/).

Zbiór Proteins jest dostępny bezpośrednio w [pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/2.6.1/index.html).

In [ ]:
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

Podobnie jak w przypadku poprzedniej sieci, także tutaj użyjemy pytorch-lightning.

W tym przypadku nie musimy implementować osobnej struktury Dataset ze względu na to, że używamy gotowego zbioru danych dostępnego i w pełni wspieranego przez pytorch-geometric.

Z tego względu, możemy przejść bezpośrednio do `LightningDataModule`.

Tak jak poprzednio dzielimy dane na zbiór treningowy, walidacyjny i testowy.

Podstawową różnicą między modułem, któego używaliśmy do obsługi danych obrazkowych jest typ używanego DataLoadera.

Wcześniej używaliśmy torchowego DataLoadera, teraz wykorzystujemy ten z pytorch-geometric.

In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
import pytorch_lightning as L
from pytorch_lightning import seed_everything
from torch.utils.data import random_split

class ProteinDataModule(L.LightningDataModule):
  def __init__(self, batch_size=32, train_size=0.8, val_size=0.1, test_size=0.1, seed=42, num_workers=3):
    super().__init__()
    seed_everything(seed, workers=True)
    self.batch_size = batch_size
    self.train_size = train_size
    self.val_size = val_size
    self.test_size = test_size
    self.num_workers = num_workers
    self.generator = torch.Generator().manual_seed(seed)

  def setup(self, stage=None):
    os.makedirs('data/TUDataset', exist_ok=True)
    dataset = TUDataset(root='data/TUDataset', name='PROTEINS', cleaned=True)
    train_size = int(self.train_size * len(dataset))
    val_size = int(self.val_size * len(dataset))
    test_size = len(dataset) - train_size - val_size
    self.train_dataset, self.val_dataset, self.test_dataset = random_split(
        dataset, [train_size, val_size, test_size], generator=self.generator)

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=True)

  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)

  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.num_workers, shuffle=False)


Przechodzimy do modelu - tutaj tworzymy go sami z operatorów dostępnych w torch-geometric.

Nasza sieć jest nieskomplikowanym GCNem z kilkoma warstwami konwolucji i zwykłą warstwą liniową z sigmoidem na końcu sieci. Używamy sigmoidu ze względu na to, że rozwiązujemy problem klasyfikacji binarnej. W przypadku wielu klas, należy użyć Softmaxa.

**Zadanie 2.1: Uzupełnij punkt 3 (3. Apply a final classifier) zgodnie z powyższym opisem. Użyj warstwy liniowej (self.linear) oraz Sigmoidu na końcu sieci.**

In [ ]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
      super(GCN, self).__init__()
      torch.manual_seed(12345)
      self.conv1 = GCNConv(num_node_features, hidden_channels)
      self.conv2 = GCNConv(hidden_channels, hidden_channels)
      self.conv3 = GCNConv(hidden_channels, hidden_channels)
      self.linear = torch.nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index, batch):
      # 1. Obtain node embeddings
      x = self.conv1(x, edge_index)
      x = x.relu()
      x = F.dropout(x, p=0.2, training=self.training)
      x = self.conv2(x, edge_index)
      x = x.relu()
      x = F.dropout(x, p=0.2, training=self.training)
      x = self.conv3(x, edge_index)

      # 2. Readout layer
      x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

      # 3. Apply a final classifier
      x = ...
      x = ...
      return x

Podobnie jak poprzednio implementujemy moduł do obsługi modelu (`LightningModule`). Tym razem jako modelu używamy GCN, jako funkcji straty - Bianry Cross Entropy, a z metryk obliczamy accuracy.

**Zadanie 2.2: Uzupełnij funkcję `_compute_acc`, aby obliczyć accuracy. Inspiracją może być tutaj funkcja `_compute_f2` z poprzedniego zadania.**

In [ ]:
class ProteinModel(L.LightningModule):
    def __init__(self, num_node_features, num_hidden, lr=1e-2):
      super().__init__()
      self.save_hyperparameters()

      self.lr = lr

      self.model = GCN(num_node_features, num_hidden)

      self.criterion = torch.nn.BCELoss()

    def forward(self, x, edge_index, batch):
      return self.model(x, edge_index, batch)

    def _compute_acc(self, y_hat, y):
      ...

    def training_step(self, batch, batch_idx):
      y_hat = self.forward(batch.x, batch.edge_index, batch.batch).float()
      y = batch.y.unsqueeze(-1).float()
      loss = self.criterion(y_hat, y)
      acc = self._compute_acc(y_hat, y)
      self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
      self.log('train_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
      return loss

    def validation_step(self, batch, batch_idx):
      y_hat = self.forward(batch.x, batch.edge_index, batch.batch).float()
      y = batch.y.unsqueeze(-1).float()
      loss = self.criterion(y_hat, y)
      acc = self._compute_acc(y_hat, y)
      self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
      self.log('val_acc', acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)

    def test_step(self, batch, batch_idx):
      y_hat = self.forward(batch.x, batch.edge_index, batch.batch).float()
      y = batch.y.unsqueeze(-1).float()
      loss = self.criterion(y_hat, y)
      acc = self._compute_acc(y_hat, y)
      self.log('test_loss', loss, logger=True)
      self.log('test_acc', acc, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(lr=self.lr, params=self.parameters())
        return optimizer

Łączymy wszystko w całość używając Trainera.

In [ ]:
import os
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer

num_node_features = TUDataset(root='data/TUDataset', name='PROTEINS').num_features

hidden_dim = 32
lr = 1e-2

# Initialize model and data module
model = ProteinModel(num_node_features, hidden_dim, lr=1e-2)
data = ProteinDataModule(batch_size=128, num_workers=2)

# Define callbacks
os.makedirs(f'data/protein/models_{hidden_dim}', exist_ok=True)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath=f'data/protein/models_{hidden_dim}',
    filename="gcn-{epoch:02d}-{val_loss:.2f}",
    save_top_k=1,
    mode="min"
)

early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="min"
)

trainer = Trainer(max_epochs=50, log_every_n_steps=7, callbacks=[checkpoint_callback, early_stopping_callback])
trainer.fit(model, datamodule=data)


In [ ]:
trainer.test(model, datamodule=data)

**Zadanie 2.3: spróbuj pozmieniać `hidden_size` i/lub `lr` (learning rate) i zobacz, jak wpływa to na jakość modelu. Możesz użyć tensorboard do porównania.**

## Przetwarzanie języka naturalnego

Spróbujemy teraz stworzyć sieć do przewidywania tematyki tekstu. Przyjmiemy następującą strategię:
- najpierw skorzystamy z gotowych word embeddings (tych samych o których mówiliśmy parę zajęć temu),
- stworzymy sieć, w której word embeddings będą tworzyć periwszą warstwę,
- nauczymy tak sklejoną sieć przewidywać jedną z 20 klas w zbiorze uczącym.

To zadanie wykonamy wyjątkowo w Keras. Tak jak zwykle, parę bibliotek na początek.

In [ ]:
%matplotlib inline

import os
import sys
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant

### Przygotowanie danych

Jako word embedding wykorzystamy model GloVe oparty o 6 miliardów tekstów. Oficjalną stroną dla tego modelu jest: https://nlp.stanford.edu/projects/glove/.

Jako zbiór danych do naszych eksperymentów wykorzystamy zbiór `newsgroup`. Jest to dosyć stary zbiór danych posiadający teksty obejmujące 20 różnych tematów. Zbiór nie jest zbyt duży dzięki czemu jest szansa nauczyć model nawet lokalnie.

Poniższy kod zawiera ścieżki, w których będziemy oczekiwać embeddings i danych. Ponadto ustawimy kilka parametrów, które wykorzystamy później.

In [ ]:
BASE_DIR = './data/'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

A teraz ściągnijmy potrzebne dane.

In [ ]:
!wget -O ./data/glove.6B.zip http://nlp.stanford.edu/data/glove.6B.zip
!wget -O ./data/news20.tar.gz http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz

Rozkakujmy paczkę z word embeddings. Zajmie sporo miejsca, ale powinna rozpakować się dosyć szybko.

In [ ]:
!unzip ./data/glove.6B.zip -d ./data/glove.6B

Warto wspomnieć, że w Internecie można znaleźć gotowe word embeddings dla różnych języków. Poniżej pierwsze dwa trafienia dla języka niemieckiego:
- https://devmount.github.io/GermanWordEmbeddings/
- https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

Dobra. Teraz przeiterujemy przez cały zestaw word embeddings i stworzymy **słownik**, gdzie kluczem będzie **słowo** a wartością wektor liczb reprezentujących **embedding**.

In [ ]:
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Super, mamy %s embeddingsów.' % len(embeddings_index))

Teraz rozpakujemy zbiór danych newsgroup. To niestety potrwa dosyć długo - choć zbiór nie jest taki duży, to składa się z wielu tysięcy plików.

In [ ]:
!tar -xzf ./data/news20.tar.gz -C ./data # to trochę potrwa

Teraz przejedziemy przez zbiór danych i wyłuskamy listę tekstów oraz wektor etykiet.

In [ ]:
texts = []  # lista tekstów (atrybuty opisowe)
labels_index = {}  # słownik mapujący klasę na liczbę (identyfikator klasy)
labels = []  # lista etykiet (atrybut decyzyjny)

for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Mamy %s tekstów.' % len(texts))

Tak się składa, że Keras oferuje własny tokenizator. Pozwoli on nam podzielić tekst na słowa i zachować tylko najpopularniejsze wyrazy (`MAX_NUM_WORDS`). Najperw odpalimy `fit`, żeby określić najpopularniejsze słowa, a następnie faktycznie stokenizujemy teksty za pomocą `texts_to_sequences`.

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Mamy %s unikatowych tokenów.' % len(word_index))

Ponieważ większość sieci lubi wejście w konkretnym rozmiarze, skorzystamy z funkcji `pad_sequences` aby ustandaryzować wejście. Za długie sekwencje słów przytniemy, a za krótkie uzupełnimy putymi znakami.

In [ ]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Jako ostatni krok przygotowania, standardowe czynności:
- kodowanie etykiet w sposób binarny,
- podział na dane uczące i walidacyjne.

In [ ]:
labels = to_categorical(np.asarray(labels))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

### Uczenie

Teraz po tym całym przetwarzaniu danych spróbujemy stworzyć klasyfikator. Najpierw wykorzystamy nasze word embeddings z Glove żeby stworzyć pierwszą warstwę sieci. Ponieważ ograniczyliśmy słownik do `MAX_NUM_WORDS`, nasza warstwa będzie miała co najwyżej tyle wag.

Pozostaje nam tylko zmapować identyfikator słowa z jego embedding, żeby zainicjalizować wagi.

In [ ]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Teraz stworzymy własną sieć. Zwróć uwagę, że korzystamy z 1-wymiarowych konwolucji, czyli "filtra" który przesuwa się po kolejnych grupach słów. Tak dla przypomnienia konwolucje 2D przesuwały kwadratowy filtr po obrazie.

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

I uczymy sieć...

In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          validation_data=(x_val, y_val))

**Zad. 3.1: Podłącz sieć do Tensorboarda i wypróbuj inne parametry lub architektury**